More info: https://www.bls.gov/tus/datafiles_0316.htm

#### 1. Read in summary

In [38]:
import os
import re
import zipfile
import pandas as pd
import numpy as np

In [30]:
test = zipfile.ZipFile(r'../data/atussum_0316.zip')
print(test.namelist())
test.close()

['atussum_0316.dat', 'Sum0316_info.txt', 'atussum_0316.sps', 'atussum_0316.sas', 'atussum_0316.do']


In [27]:
%%time
data = {}
counter = 0
with zipfile.ZipFile(r'../data/atussum_0316.zip') as myzip:
    with myzip.open('atussum_0316.dat') as myfile:
        header = myfile.readline().decode('ascii').strip().split(',')
        for line in myfile:
            data[counter] = line.decode('ascii').strip().split(',')
            counter+=1

Wall time: 6.89 s


In [37]:
print(len(header))
header[:10]

455


['TUCASEID',
 'GEMETSTA',
 'GTMETSTA',
 'PEEDUCA',
 'PEHSPNON',
 'PTDTRACE',
 'TEAGE',
 'TELFS',
 'TEMJOT',
 'TESCHENR']

In [39]:
np.repeat([np.int],8)

array([<class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>,
       <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>], dtype=object)

In [47]:
df = pd.DataFrame.from_dict(data, orient='index')
df.columns = header
df[:3]

,TUCASEID,GEMETSTA,GTMETSTA,PEEDUCA,PEHSPNON,PTDTRACE,TEAGE,TELFS,TEMJOT,TESCHENR,...,t181801,t181899,t189999,t500101,t500103,t500104,t500105,t500106,t500107,t509989
0,20030100013280,1,-1,44,2,2,60,2,2,-1,...,0,0,0,0,0,0,0,0,0,0
1,20030100013344,2,-1,40,2,1,41,1,2,2,...,0,0,0,0,0,0,0,0,0,0
2,20030100013352,1,-1,41,2,1,26,2,2,2,...,0,0,0,0,0,0,0,0,0,0


In [48]:
df['t010102'].describe() # sleeplessness # note the data type

count     181335
unique       308
top            0
freq      173193
Name: t010102, dtype: object

In [49]:
df = df.convert_objects(convert_numeric=True)
df['t010102'].describe()

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


count    181335.000000
mean          3.689084
std          25.352866
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max        1365.000000
Name: t010102, dtype: float64

In [54]:
# Collapse to 4 digit level
activity_codes = [c for c in df.columns if re.match("t[0-9]{6,}",c) is not None]
other_vars = [c for c in df.columns if re.match("t[0-9]{6,}",c) is None]
print(len(activity_codes),len(other_vars))

431 24


In [58]:
print(df.shape, df['TUCASEID'].nunique()) # TUCASEID uniquely identifies things

(181335, 455) 181335


In [59]:
df_oth = df[other_vars]
df_act = df[['TUCASEID']+activity_codes]

In [67]:
df_act_t = df_act.T.reset_index()
df_act_t.columns = df_act_t.iloc[0]
df_act_t = df_act_t.iloc[1:,:]
df_act_t[:3]

,TUCASEID,20030100013280,20030100013344,20030100013352,20030100013848,20030100014165,20030100014169,20030100014209,20030100014427,20030100014550,...,20161212162045,20161212162065,20161212162091,20161212162212,20161212162217,20161212162307,20161212162357,20161212162426,20161212162456,20161212162509
1,t010101,870,620,560,720,385,675,531,445,645,...,473,650,710,540,490,520,645,402,420,910
2,t010102,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,t010199,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
df_act_t['ACTCODE_4D'] = df_act_t['TUCASEID'].map(lambda x: str(x)[:5])
df_act_t[:3]

,TUCASEID,20030100013280,20030100013344,20030100013352,20030100013848,20030100014165,20030100014169,20030100014209,20030100014427,20030100014550,...,20161212162065,20161212162091,20161212162212,20161212162217,20161212162307,20161212162357,20161212162426,20161212162456,20161212162509,ACTCODE_4D
1,t010101,870,620,560,720,385,675,531,445,645,...,650,710,540,490,520,645,402,420,910,t0101
2,t010102,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,t0101
3,t010199,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,t0101


In [70]:
df_act_4d = df_act_t.groupby('ACTCODE_4D').sum().T.reset_index()
df_act_4d[:3]

ACTCODE_4D,0,t0101,t0102,t0103,t0104,t0105,t0199,t0201,t0202,t0203,...,t1811,t1812,t1813,t1814,t1815,t1816,t1818,t1899,t5001,t5099
0,20030100013280,870,40,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20030100013344,620,60,0,0,0,0,0,0,0,...,0,20,0,0,0,0,0,0,0,0
2,20030100013352,560,80,0,0,0,0,15,240,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
df_act_4d_all = pd.merge(df_oth, df_act_4d, 
                         how='inner', left_on='TUCASEID', right_on=0)
df_act_4d_all[:3]

,TUCASEID,GEMETSTA,GTMETSTA,PEEDUCA,PEHSPNON,PTDTRACE,TEAGE,TELFS,TEMJOT,TESCHENR,...,t1811,t1812,t1813,t1814,t1815,t1816,t1818,t1899,t5001,t5099
0,20030100013280,1,-1,44,2,2,60,2,2,-1,...,0,0,0,0,0,0,0,0,0,0
1,20030100013344,2,-1,40,2,1,41,1,2,2,...,0,20,0,0,0,0,0,0,0,0
2,20030100013352,1,-1,41,2,1,26,2,2,2,...,0,0,0,0,0,0,0,0,0,0


In [73]:
del data
del df
del df_act
del df_act_t
del df_oth

In [72]:
df_act_4d_all.to_csv(r"../data/converted/summary_4d.csv",index=False)

#### 2. Read in resp

In [75]:
%%time
data = {}
counter = 0
with zipfile.ZipFile(r'../data/atusresp_0316.zip') as myzip:
    with myzip.open('atusresp_0316.dat') as myfile:
        header = myfile.readline().decode('ascii').strip().split(',')
        for line in myfile:
            data[counter] = line.decode('ascii').strip().split(',')
            counter+=1
print(len(data))

181335
Wall time: 53.4 s


In [78]:
df = pd.DataFrame.from_dict(data, orient='index')
df = df.convert_objects(convert_numeric=True)
df.columns = header
df[:3]

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


,TUCASEID,TULINENO,TESPUHRS,TRDTIND1,TRDTOCC1,TRERNHLY,TRERNUPD,TRHERNAL,TRHHCHILD,TRIMIND1,...,TRYHHCHILD,TRWBMODR,TRTALONE_WK,TRTCCC_WK,TRLVMODR,TRTEC,TUECYTD,TUELDER,TUELFREQ,TUELNUM
0,20030100013280,1,-1.0,40,8,2200.0,1,1,2,15,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,20030100013344,1,50.0,16,16,-1.0,1,-1,1,5,...,0,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,20030100013352,1,-1.0,43,15,1250.0,0,0,2,16,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
